# Project 2: Breakout Strategy

This project is meant to test some a Breakout Strategy for some crypto assets. It generates signals based on the high and low bounds of trading an asset and test the significance of the returns to understand the viability of the strategy. For more questions please email me at hosammhmahmoud@gmail.com 

### Step 1 Extract Data and Plot high, low, close figures

In [159]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sqlalchemy import create_engine
from tqdm import tqdm
import numpy as np 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go


api_key = '61fd48a1d098f28f6015d7283be5477c06f040f87badec4b03f592b0c9bbaeb0'


def CCData_OHLCV(fsym, tsym, limit, exchange='CCCAGG', freq='day'):
    """ 
    Get the price of any pair on certin exchange for certin frequency 
    
    Parameteres
    -----------
    fsym : string > from symbol 
    tsym : string > to symbol 
    limit : int > number of frequency required
    exchange : string >  Exchange you need data from
    
    Returns 
    -------
    df : DataFrame 
      OHLCV dataframe for the pair required 
    """
    url = (f'https://min-api.cryptocompare.com/data/v2/histo{freq}?'
           f'fsym={fsym}&tsym={tsym}&limit={limit}&e={exchange}&'
           f'tryConversion=true&api_key={api_key}')
    df = pd.DataFrame(requests.get(url).json()['Data']['Data'])
    df.index = pd.to_datetime(df['time'], unit='s')
    return df

btc_ohlcv = CCData_OHLCV('btc', 'usd', '360')

fig = px.line(btc_ohlcv, x = btc_ohlcv.index, y=['high', 'low','close'], title = 'BTC Close High and Low')
fig.show()


In this project, we assume that the first three steps of Alpha Research done ("observe & research", "form hypothesis", "validate hypothesis"). The hypothesis  for this project is the following:
- In the absence of news or significant investor trading interest, stocks oscillate in a range.
- Traders seek to capitalize on this range-bound behaviour periodically by selling/shorting at the top of the range and buying/covering at the bottom of the range. This behaviour reinforces the existence of the range.
- When stocks break out of the range, due to, e.g., a significant news release or from market pressure from a large investor:
    - the liquidity traders who have been providing liquidity at the bounds of the range seek to cover their positions to mitigate losses, thus magnifying the move out of the range, _and_
    - the move out of the range attracts other investor interest; these investors, due to the behavioural bias of _herding_ (e.g., [Herd Behavior](https://www.investopedia.com/university/behavioral_finance/behavioral8.asp)) build positions which favor continuation of the trend.

### Step 2: Get moving averages of Highs and lows for a certin window

In [203]:
def get_moving_average_highs_lows(high, low, window):
    """
    Get the highs and lows in a lookback window.
    
    Parameters
    ----------
    high : DataFrame
        High price for each ticker and date
    low : DataFrame
        Low price for each ticker and date
    lookback_days : int
        The number of days to look back
    
    Returns
    -------
    lookback_high : DataFrame
        Lookback high price for each ticker and date
    lookback_low : DataFrame
        Lookback low price for each ticker and date
    """
    ma_high = high.shift(1).rolling(window=window).max()
    ma_low = low.shift(1).rolling(window=window).min()
    return ma_high, ma_low

window = 7
btc_rolling_high, btc_rolling_low = get_moving_average_highs_lows(btc_ohlcv['high'], btc_ohlcv['low'], window)

fig = px.line(btc_ohlcv['close'], title = f'BTC Close High and Low with winow of {window} days'.format())
fig.add_trace(px.line(btc_rolling_high.to_frame(),  color_discrete_sequence=['green']).data[0])
fig.add_trace(px.line(btc_rolling_low.to_frame(), color_discrete_sequence=['red']).data[0])

fig.show()

### Step 3: Compute Long and Short Signals

Using the generated indicator of highs and lows, create long and short signals using a breakout strategy.

| Signal | Condition |
|----|------|
| -1 | Low > Close Price |
| 1  | High < Close Price |
| 0  | Otherwise |

In [204]:

def get_long_short(close, lookback_high, lookback_low): 
    """
    Get signals from comparing the close and high and low dataframes
    
    Parameters
    ----------
    close : DataFrame
        close price for asset
    lookback_high : DataFrame
        moving average high prices
    lookback_low : int
        moving average low prices
    
    Returns
    -------
    signals: Series 
        series of 1, -1 or 0 signals which translates to long, short or do nothing
    """

    signals = np.where(close > lookback_high, 1, np.where(close < lookback_low, -1, 0))
    
    return pd.Series(signals, index = btc_ohlcv.index, name='signals')


signals = get_long_short(btc_ohlcv['close'], btc_rolling_high, btc_rolling_low)

fig = px.line(btc_ohlcv['close'], title = 'BTC Close High and Low')
fig.add_trace(px.scatter((signals.loc[signals ==1]*btc_ohlcv['close']), 
                         color_discrete_sequence=['green']).data[0])
fig.add_trace(px.scatter(signals.loc[signals ==-1]*-btc_ohlcv['close'], 
                         color_discrete_sequence=['red']).data[0])
fig.show()


### Step 4: Filter signals 

That was a lot of repeated signals! If we're already shorting a stock, having an additional signal to short a stock isn't helpful for this strategy. This also applies to additional long signals when the last signal was long.


In [157]:
def filter_signals(signal):
    """
    Filter out signals in a Series so no long comes after long and vice versa
    
    Parameters
    ----------
    signal : DataFrame
        The long, short, and do nothing signals
    
    Returns
    -------
    filtered_signal : DataFrame
        The filtered long, short, and do nothing
    """
    prev_value = None
    for idx, value in enumerate(signal):
        if value == prev_value: 
            signal.iloc[idx] = 0
        if value != 0: 
            prev_value = value 
    
    return signal

filtered_signals = filter_signals(signals)

fig = px.line(btc_ohlcv['close'], title = 'BTC Close High and Low')
fig.add_trace(px.scatter((filtered_signals.loc[signals ==1]*btc_ohlcv['close']), 
                         color_discrete_sequence=['green']).data[0])
fig.add_trace(px.scatter(filtered_signals.loc[signals ==-1]*-btc_ohlcv['close'], 
                         color_discrete_sequence=['red']).data[0])
fig.show()


### Step 5: Compute the Signal Return 

Using the price returns generate the signal returns.

In [198]:
def calculate_returns(close, filtered_signals, method="short_long"): 
    returns = close.pct_change()
    df = filtered_signals.replace(0, np.nan).fillna(method='ffill')
    
    
    if method == 'only_long':
            df = df.replace(-1, 0)

    return df * returns 

returns = calculate_returns(btc_ohlcv['close'], filtered_signals)
returns_only_long = calculate_returns(btc_ohlcv['close'], filtered_signals, method = 'only_long')


trace1 = go.Scatter(x=btc_ohlcv.index, y=btc_ohlcv['close'], mode='lines', name='BTC Close', yaxis='y1')
trace2 = go.Scatter(x=returns.index, y=returns, mode='lines', name='Returns', line=dict(color='black'), yaxis='y2')
fig = go.Figure(data=[trace1, trace2])
fig.update_layout( title='BTC Close and Returns',
                    xaxis_title='Date',
                    yaxis_title='BTC Close',
                    yaxis2=dict( title='Returns', overlaying='y', side='right'))
fig.show()

trace1 = go.Scatter(x=btc_ohlcv.index, y=btc_ohlcv['close'], mode='lines', name='BTC Close', yaxis='y1')
trace2 = go.Scatter(x=returns_only_long.index, y=returns_only_long, mode='lines', name='Returns', line=dict(color='black'), yaxis='y2')
fig = go.Figure(data=[trace1, trace2])
fig.update_layout( title='BTC Close and Returns - Long Only Portfolio',
                    xaxis_title='Date',
                    yaxis_title='BTC Close',
                    yaxis2=dict( title='Returns', overlaying='y', side='right'))
fig.show()



### Step 6: Check the significance of the returns using normal distributions p_values 

- i p_value < 0.05: Returns are significant - the strategy works 
-  if p_valye > 0.05: our results aren't significant 

In [214]:
import scipy.stats as stats
 
trace1 = go.Histogram(x=returns, nbinsx=30, name='Returns')
trace2 = go.Histogram(x=returns_only_long, nbinsx=30, name='Returns (Long)')

fig = make_subplots(rows=1, cols=2, subplot_titles=['Returns', 'Returns (Long Only)'])
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)

fig.update_layout(title='Histograms of Portfolio Returns', showlegend=False)
fig.show()
 
# ----- scipy.stats test ------- 

alpha = 0.05

t_statistic, p_value = stats.ttest_1samp(returns.dropna(), 0)
t_statistic_long, p_value_long = stats.ttest_1samp(returns_only_long.dropna(), 0)

if p_value < alpha:
    print(f'pvalue = {p_value}'.format())
    print("Reject the null hypothesis: There is a significant difference in long returns from 0.")
else:
    print(f'pvalue = {p_value}'.format())
    print("Fail to reject the null hypothesis: There is no significant difference in long returns from 0.")

if p_value_long < alpha:
    print(f'pvalue_long = {p_value_long}'.format())
    print("Reject the null hypothesis: There is a significant difference in long returns from 0.")
else:
    print(f'pvalue = {p_value_long}'.format())
    print("Fail to reject the null hypothesis: There is no significant difference in long returns from 0.")




pvalue = 0.0003044398804993109
Reject the null hypothesis: There is a significant difference in long returns from 0.
pvalue_long = 0.001597605665315448
Reject the null hypothesis: There is a significant difference in long returns from 0.


The project is done 
